In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re
import lxml.html as lh

In [2]:
html_page = requests.get('https://en.wikipedia.org/wiki/User:Michael_J/County_table')
doc = lh.fromstring(html_page.content)

In [3]:
tr_elements = doc.xpath('//tr')

In [4]:
[len(T) for T in tr_elements[:12]]

[14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14]

In [5]:
col = []
i = 0 

for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Sort [1]"
2:"State"
3:"FIPS"
4:"County [2]"
5:"County Seat(s) [3]"
6:"Population(2010)"
7:"Land Areakm²"
8:"Land Areami²"
9:"Water Areakm²"
10:"Water Areami²"
11:"Total Areakm²"
12:"Total Areami²"
13:"Latitude"
14:"Longitude
"


In [6]:
for j in range(1,len(tr_elements)):
    # Original code from https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059
    T=tr_elements[j]
    
    if len(T)!=14:
        break
    i=0
    for t in T.iterchildren():
        data=t.text_content() 
        
        if i>0:
            try:
                data=int(data)
            except:
                pass
        
        col[i][1].append(data)
        i+=1

In [7]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df.head()

,Sort [1],State,FIPS,County [2],County Seat(s) [3],Population(2010),Land Areakm²,Land Areami²,Water Areakm²,Water Areami²,Total Areakm²,Total Areami²,Latitude,Longitude\n
0,1,AL,1001,Autauga,Prattville,"54,571","1,539.582",594.436,25.776,9.952,"1,565.358",604.388,+32.536382°,–86.644490°\n
1,2,AL,1003,Baldwin,Bay Minette,"182,265","4,117.522","1,589.784","1,133.190",437.527,"5,250.712","2,027.311",+30.659218°,–87.746067°\n
2,3,AL,1005,Barbour,Clayton,"27,457","2,291.819",884.876,50.865,19.639,"2,342.684",904.515,+31.870670°,–85.405456°\n
3,4,AL,1007,Bibb,Centreville,"22,915","1,612.481",622.582,9.289,3.587,"1,621.770",626.169,+33.015893°,–87.127148°\n
4,5,AL,1009,Blount,Oneonta,"57,322","1,669.962",644.776,15.157,5.852,"1,685.119",650.628,+33.977448°,–86.567246°\n


In [8]:
to_drop = ['Sort [1]', 'County Seat(s) [3]', 'Land Areakm²', 'Water Areakm²', 'Total Areakm²']
df.drop(to_drop, axis=1, inplace=True)
df.rename(columns={'County [2]' : 'County', 'Longitude\n' : 'Longitude',
                   'Land Areami²': 'Land Area mi²', 'Water Areami²' : 'Water Area mi²',
                   'Total Areami²' : 'Total Area mi²'}, inplace=True)
df.head()

,State,FIPS,County,Population(2010),Land Area mi²,Water Area mi²,Total Area mi²,Latitude,Longitude
0,AL,1001,Autauga,"54,571",594.436,9.952,604.388,+32.536382°,–86.644490°\n
1,AL,1003,Baldwin,"182,265","1,589.784",437.527,"2,027.311",+30.659218°,–87.746067°\n
2,AL,1005,Barbour,"27,457",884.876,19.639,904.515,+31.870670°,–85.405456°\n
3,AL,1007,Bibb,"22,915",622.582,3.587,626.169,+33.015893°,–87.127148°\n
4,AL,1009,Blount,"57,322",644.776,5.852,650.628,+33.977448°,–86.567246°\n


In [9]:
df.Longitude = ['-' + df.Longitude[i][1:-2] for i in range(len(df.Longitude))]
df.Latitude = [df.Latitude[i][1:-1] for i in range(len(df.Latitude))]
df.County = [df.County[i].title() for i  in range(len(df.County))]
for i in range(len(df.County)):
    df.County.iloc[i] = df.County.iloc[i].replace('St.', 'Saint')

C:\Users\helde\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3143 entries, 0 to 3142
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   State             3143 non-null   object
 1   FIPS              3143 non-null   int64 
 2   County            3143 non-null   object
 3   Population(2010)  3143 non-null   object
 4   Land Area mi²     3143 non-null   object
 5   Water Area mi²    3143 non-null   object
 6   Total Area mi²    3143 non-null   object
 7   Latitude          3143 non-null   object
 8   Longitude         3143 non-null   object
dtypes: int64(1), object(8)
memory usage: 221.1+ KB


In [11]:
cols_to_float = ['Longitude', 'Latitude']
for col in cols_to_float:
    df[col] = df[col].astype(float)

In [12]:
df['State and County'] = df.State + ' ' + df.County + ' County'

In [13]:
new_df = df[['State and County', 'FIPS', 'Longitude', 'Latitude', 'Population(2010)', 'Total Area mi²']]
new_df.head()

,State and County,FIPS,Longitude,Latitude,Population(2010),Total Area mi²
0,AL Autauga County,1001,-86.644490,32.536382,"54,571",604.388
1,AL Baldwin County,1003,-87.746067,30.659218,"182,265","2,027.311"
2,AL Barbour County,1005,-85.405456,31.870670,"27,457",904.515
3,AL Bibb County,1007,-87.127148,33.015893,"22,915",626.169
4,AL Blount County,1009,-86.567246,33.977448,"57,322",650.628


In [14]:
new_df.to_csv('../data/County_cords.csv', index=False)

In [15]:
this_df = pd.read_csv('../data/County_cords.csv')
this_df.head()

,State and County,FIPS,Longitude,Latitude,Population(2010),Total Area mi²
0,AL Autauga County,1001,-86.644490,32.536382,"54,571",604.388
1,AL Baldwin County,1003,-87.746067,30.659218,"182,265","2,027.311"
2,AL Barbour County,1005,-85.405456,31.870670,"27,457",904.515
3,AL Bibb County,1007,-87.127148,33.015893,"22,915",626.169
4,AL Blount County,1009,-86.567246,33.977448,"57,322",650.628


In [23]:
this_df['Total Area mi²'].iloc[0][:3]

'604'